<h1>different loaders for loading data</h1>

In [1]:
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
text_loader = TextLoader("conversational_RAG_theory.txt");

data = text_loader.load()

In [3]:
# print(data[0].metadata)
# print(data[0])

In [4]:
from langchain_community.document_loaders import PyPDFLoader


In [5]:
loader = PyPDFLoader("2024_Siddhesh_Sonawane_pod.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [6]:
# print(pages[0])
# print(pages[1])
# print(f"{pages[0].metadata}\n")
# print(pages[0].page_content)

In [7]:
from langchain_community.document_loaders import UnstructuredURLLoader


In [8]:
urls = [
    "https://www.moneycontrol.com/news/telecom/vodafone-idea-bets-big-on-gaming-content-to-widen-4g-subscriber-base-12832202.html",
    "https://www.moneycontrol.com/news/business/moneycontrol-research/craftsman-automation-significant-headroom-to-grow-valuation-at-fair-level-12832073.html",
]
loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

len(data)

2

In [9]:
data[0].metadata

{'source': 'https://www.moneycontrol.com/news/telecom/vodafone-idea-bets-big-on-gaming-content-to-widen-4g-subscriber-base-12832202.html'}

In [10]:
data[1].metadata

{'source': 'https://www.moneycontrol.com/news/business/moneycontrol-research/craftsman-automation-significant-headroom-to-grow-valuation-at-fair-level-12832073.html'}

<h1>using different techniques for text splitter</h1>

In [11]:
# from langchain.text_splitter import CharacterTextSplitter

# splitter = CharacterTextSplitter(separator = "\n", chunk_size = 200, chunk_overlap = 0)
# chunks = splitter.split_text(data[0].page_content)

In [12]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# r_splitter = RecursiveCharacterTextSplitter(separators = ["\n\n","\n"," "], chunk_size = 200, chunk_overlap = 0)

# docs = chunks = r_splitter.split_text(data[0].page_content)
# len(docs)


In [13]:
# for chunk in chunks:
#     print(str(len(chunk)))


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [15]:
len(docs)

15

In [16]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/telecom/vodafone-idea-bets-big-on-gaming-content-to-widen-4g-subscriber-base-12832202.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_NEWS_AS/MC_ENG_ROS_NWS_AS_ATF_728\n\nGo PRO @₹99 PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewstelecomVodafone Idea bets big on gaming, content to widen 4G subscriber base\n\nVodafone Idea bets big

<h1>embedding the data collected from url into vector data</h1>

In [17]:
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

In [20]:
from langchain.vectorstores import FAISS
from langchain_mistralai.embeddings import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(model="mistral-embed")

# creating vector store
vectorindex_mistral = FAISS.from_documents(docs, embeddings)


In [22]:
retriever = vectorindex_mistral.as_retriever()

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [28]:
response = retrieval_chain.invoke({"input": "explain articles"})
print(response["answer"])

Based on the provided context, here are explanations of the articles mentioned:

1. **Trending news articles:**
   - **"Microsoft CEO Satya Nadella: AI can help you be more productive at work"** - This article likely discusses Satya Nadella's views on how artificial intelligence can enhance productivity in the workplace.
   - **"Half of HR team fired after employee exposes glitch rejecting job applications instantly. Viral post"** - This article probably covers a viral incident where an employee revealed a glitch in the HR system that led to the dismissal of half the HR team.
   - **"Only 33% of American millionaires think they are rich, finds study"** - This article presents findings from a study about American millionaires' perceptions of their wealth.
   - **"Bengaluru AI startup co-founder's 18-hour workdays spark debate: 'His bed is 3-feet from computer'"** - This article discusses the debate surrounding the intense work schedule of a Bengaluru AI startup co-founder.
   - **"Furio

In [29]:
response = retrieval_chain.invoke({"input": "what is nazra tech"})
print(response["answer"])

Nazra Tech is a company that has completed a Rs 150 crore investment in PokerBaazi parent Moonshine Technology.
